# Dataset Info

In [22]:
import sys, os
import json
import matplotlib.pyplot as plt

In [23]:
data_folder = "/Users/suser/Courses/Semantic Parsing/datasets/SNIPS"

def add_spaces(sent):
    ret = sent
    for char in [',', '.', '\'', '?', ':', '-', '!', '`', '…', '&', '’', '/', '¡', '–', '"', "“", ';', '=', '”', 
                "~", '—', "%"]:
        ret = ret.replace(char, ' ' + char + ' ')
    return ret
    
intents = ["PlayMusic", "RateBook", "SearchCreativeWork", "GetWeather", "BookRestaurant", "AddToPlaylist", "SearchScreeningEvent"]
print('Intents: ' + ' '.join(intents) + '\n')
train_data = {k : {} for k in intents}
val_data = {k: {} for k in intents}

for intent in intents:
    train_file = os.path.join(data_folder, intent, 'train_{}_full.json'.format(intent))
    train_json = json.load(open(train_file, 'rb'))
    train_utterances = []
    train_entities = []
    for i in range(len(train_json[intent])):
        utt = ' '.join(add_spaces(' '.join([a['text'] for a in train_json[intent][i]['data']])).split())
        ets = [(' '.join(add_spaces(a['text']).split()), a['entity']) for a in train_json[intent][i]['data'] if 'entity' in a]
        
        utts = tokenizer(utt, return_tensors = "pt", add_special_tokens=False)
        if len(utt.split()) != max(utts.word_ids()) + 1:
            continue
            
        train_utterances.append(utt)
        train_entities.append(ets)
        
    train_data[intent]['utterances'] = train_utterances
    train_data[intent]['entities'] = train_entities
    
    val_file = os.path.join(data_folder, intent, 'validate_{}.json'.format(intent))
    val_json = json.load(open(val_file, 'rb'))
    val_utterances = [' '.join(add_spaces(' '.join([a['text'] for a in val_json[intent][i]['data']])).split()) for i in range(len(val_json[intent]))]
    val_entities = [[(' '.join(add_spaces(a['text']).split()), a['entity']) for a in val_json[intent][i]['data'] if 'entity' in a] for i in range(len(val_json[intent]))]
    val_data[intent]['utterances'] = val_utterances
    val_data[intent]['entities'] = val_entities
    
for intent in intents:
    slots = list(set([b[1] for a in train_data[intent]['entities'] for b in a ]))
    span_lengths = [(len(b[0].split()), b[0]) for a in train_data[intent]['entities'] for b in a ]
    span_lengths.sort(key = lambda x: x[0], reverse=True)
    print('{}, \ttrain: {}, \tval: {}, \tnum slots: {}, \tspan lengths: [{}, {}]'.format(intent,
                                                                        len(train_data[intent]['utterances']),
                                                                        len(val_data[intent]['utterances']),
                                                                        len(slots),
                                                                        span_lengths[-1][0],
                                                                        span_lengths[0][0]))
    print('\t' + '\n\t'.join(slots).replace('_', ' ') + '\n')
    

Intents: PlayMusic RateBook SearchCreativeWork GetWeather BookRestaurant AddToPlaylist SearchScreeningEvent

PlayMusic, 	train: 2000, 	val: 100, 	num slots: 9, 	span lengths: [1, 9]
	artist
	year
	service
	sort
	music item
	playlist
	genre
	track
	album

RateBook, 	train: 1956, 	val: 100, 	num slots: 7, 	span lengths: [1, 20]
	object type
	object select
	rating unit
	object name
	object part of series type
	best rating
	rating value

SearchCreativeWork, 	train: 1953, 	val: 100, 	num slots: 2, 	span lengths: [1, 23]
	object type
	object name

GetWeather, 	train: 2000, 	val: 100, 	num slots: 9, 	span lengths: [1, 12]
	condition temperature
	spatial relation
	geographic poi
	condition description
	current location
	city
	timeRange
	state
	country

BookRestaurant, 	train: 1973, 	val: 100, 	num slots: 14, 	span lengths: [1, 9]
	party size number
	spatial relation
	served dish
	restaurant name
	party size description
	cuisine
	city
	poi
	state
	restaurant type
	timeRange
	facility
	sort
	cou

# Rate Book: Span Analysis

In [388]:
intent = 'RateBook'
span_lengths = [len(b[0].split()) for a in train_data[intent]['entities'] for b in a ]
span_lengths.sort(reverse=True)
print(len(span_lengths))
plt.hist(span_lengths, bins=17)
plt.title('Span length counts')
plt.show()

7349


In [389]:
ex_lengths = [[len(b[0].split()) for b in a] for a in train_data[intent]['entities']]
ex_max_lengths = [max(a) for a in ex_lengths]
plt.hist(ex_max_lengths, bins=17)
plt.title('Max span length per utterance counts')
plt.show()

# Negative example construction

In [391]:
def find_all_spans(words, threshold):
    all_spans = []
    span_ids = []
    for i in range(1, threshold):
        for j in range(0, len(words) - i + 1):
            all_spans.append(' '.join(words[j:j+i]))
            span_ids.append((j, j+i))
    return all_spans, span_ids

def is_overlap(a, b):
    aw = a.split()
    bw = b.split()
    for w in bw:
        if w in aw:
            return True
    return False

span_threshold = 6

train_entity_data = {i: [] for i in intents}

for intent in intents:
    for i in range(len(train_data[intent]["utterances"])):
        utt = train_data[intent]["utterances"][i]
        ets = train_data[intent]["entities"][i]
        all_spans, span_ids = find_all_spans(utt.split(), span_threshold)
        
        ets_ids = []
        for et in ets:
            if et[0] in all_spans:
                ets_ids.append(span_ids[all_spans.index(et[0])])

        skip_ex = False
        for et in ets:
            if et[0] not in all_spans:
                skip_ex = True
            else:
                rm_idx = all_spans.index(et[0])
                all_spans.remove(et[0])
                span_ids = span_ids[0:rm_idx] + span_ids[rm_idx+1:]

        if skip_ex:
            continue

        for et, eid in zip(ets, ets_ids):
            name = et[1]
            pos = (et[0], eid)
            neg_ns = [(s, i) for (s, i) in zip(all_spans, span_ids) if not is_overlap(pos[0], s)]
            neg_os = [(s, i) for (s, i) in zip(all_spans, span_ids) if is_overlap(pos[0], s)]
            neg_oe = [(b[0], i) for (b, i) in zip(ets, ets_ids) if b[1] != name]
            train_entity_data[intent].append({'name': name,
                                     'utterance': utt,
                                     'positive': pos,
                                     'negative_non_overlap_spans': neg_ns,
                                     'negative_overlap_spans': neg_os,
                                     'negative_other_entites': neg_oe})

In [392]:
train_entity_data['PlayMusic'][50]

{'name': 'year',
 'utterance': "I ' d like to hear music from 1951 , preferably a soundtrack .",
 'positive': ('1951', (8, 9)),
 'negative_non_overlap_spans': [('I', (0, 1)),
  ("'", (1, 2)),
  ('d', (2, 3)),
  ('like', (3, 4)),
  ('to', (4, 5)),
  ('hear', (5, 6)),
  ('music', (6, 7)),
  ('from', (7, 8)),
  (',', (9, 10)),
  ('preferably', (10, 11)),
  ('a', (11, 12)),
  ('.', (13, 14)),
  ("I '", (0, 2)),
  ("' d", (1, 3)),
  ('d like', (2, 4)),
  ('like to', (3, 5)),
  ('to hear', (4, 6)),
  ('hear music', (5, 7)),
  ('music from', (6, 8)),
  (', preferably', (9, 11)),
  ('preferably a', (10, 12)),
  ('a soundtrack', (11, 13)),
  ('soundtrack .', (12, 14)),
  ("I ' d", (0, 3)),
  ("' d like", (1, 4)),
  ('d like to', (2, 5)),
  ('like to hear', (3, 6)),
  ('to hear music', (4, 7)),
  ('hear music from', (5, 8)),
  (', preferably a', (9, 12)),
  ('preferably a soundtrack', (10, 13)),
  ('a soundtrack .', (11, 14)),
  ("I ' d like", (0, 4)),
  ("' d like to", (1, 5)),
  ('d like to he

# Margin Loss Dataset Creation

In [393]:
from random import sample, shuffle

margin_train_data = {i: [] for i in intents}

for intent in intents:
    for et in train_entity_data[intent]:
        utt = et["utterance"]
        name = et["name"].replace('_', " ")
        pos = et["positive"]
        min_ns = min(len(et["negative_non_overlap_spans"]), 5)
        min_os = min(len(et["negative_overlap_spans"]), 5)
        negs = (et["negative_other_entites"] + sample(et["negative_overlap_spans"], min_os) 
                + sample(et["negative_non_overlap_spans"], min_ns))
        for neg in negs:
            margin_train_data[intent].append((name, pos, neg, utt))
    shuffle(margin_train_data[intent])

In [394]:
margin_train_data['PlayMusic'][:25]

[('music item',
  ('movement', (5, 6)),
  ('Play me', (0, 2)),
  'Play me a Mario Vazquez movement'),
 ('music item',
  ('concerto', (1, 2)),
  ('concerto from the eighties', (1, 5)),
  'Play concerto from the eighties'),
 ('artist',
  ('Skeets Mcdonald', (8, 10)),
  ('hear Box Of Rain', (3, 7)),
  'I want to hear Box Of Rain by Skeets Mcdonald'),
 ('service',
  ('Netflix', (5, 6)),
  ('Don Ho', (2, 4)),
  'Play some Don Ho on Netflix'),
 ('artist',
  ('Abderrahmane Abdelli', (9, 11)),
  ('Impossible Is Nothing', (5, 8)),
  'Do you have something like Impossible Is Nothing by Abderrahmane Abdelli ?'),
 ('service',
  ('Netflix', (4, 5)),
  ('David Banner', (1, 3)),
  'play David Banner on Netflix'),
 ('artist',
  ('Monie Love', (1, 3)),
  ('song', (11, 12)),
  'play Monie Love tunes on Google Music from 1984 from her song list'),
 ('music item',
  ('soundtrack', (3, 4)),
  ('soundtrack by', (3, 5)),
  'Play a seventies soundtrack by steve walsh on Groove Shark'),
 ('genre',
  ('alternat

In [395]:
train_entity_data.keys()

dict_keys(['PlayMusic', 'RateBook', 'SearchCreativeWork', 'GetWeather', 'BookRestaurant', 'AddToPlaylist', 'SearchScreeningEvent'])

In [396]:
import pickle

data_folder = "/Users/suser/Courses/Semantic Parsing/datasets/SNIPS/processed"
pickle.dump(train_entity_data, open(os.path.join(data_folder, 'train_entity_data.p'), 'wb'))
pickle.dump(margin_train_data, open(os.path.join(data_folder, 'margin_train_data.p'), 'wb'))

In [397]:
pickle.dump(train_data, open(os.path.join(data_folder, 'train_data.p'), 'wb'))
pickle.dump(val_data, open(os.path.join(data_folder, 'val_data.p'), 'wb'))

In [535]:
import pickle
import math

ho = 'GetWeather'
eo = 'GetWeather'
model = 'base'
k = 1

metrics = pickle.load(open('metrics/ho_{}_ev_{}_{}_metrics.p'.format(ho, eo, model), 'rb'))

print('Trained without: {}'.format(ho))
print('Evaluated on : {}'.format(eo))
print()
print('Top {} accuracy'.format(k))
for i in metrics:
    if i == 'all':
        continue
    else:
        total = 0
        count = 0
        for r in metrics[i]:
            total += 1
            if (r < k) and (r != -1):
                count += 1
        print('\t{}: {:.2f}%'.format(i.upper(),count/total*100))
        
print()
total = 0
count = 0
for r in metrics['all']:
    total += 1
    if (r < k) and (r != -1):
        count += 1
print('\t{}: {:.2f}%'.format('OVERALL',count/total*100))
print()

print('Mean Reciprocal Rank')
for i in metrics:
    if i == 'all':
        continue
    else:
        mrr = []
        for r in metrics[i]:
            if r == -1:
                mrr.append(0)
            else:
                mrr.append(1/(r+1))
        print('\t{}: {:.4f}'.format(i.upper(), sum(mrr)/len(mrr)))
        
print()
mrr = []
for r in metrics['all']:
    if r == -1:
        mrr.append(0)
    else:
        mrr.append(1/(r+1))
print('\t{}: {:.4f}'.format('OVERALL', sum(mrr)/len(mrr)))
print()


Trained without: GetWeather
Evaluated on : GetWeather

Top 1 accuracy
	STATE: 100.00%
	SPATIAL RELATION: 100.00%
	CONDITION DESCRIPTION: 9.09%
	COUNTRY: 100.00%
	TIMERANGE: 95.31%
	CITY: 94.74%
	CONDITION TEMPERATURE: 47.62%
	CURRENT LOCATION: 17.65%
	GEOGRAPHIC POI: 81.25%

	OVERALL: 78.10%

Mean Reciprocal Rank
	STATE: 1.0000
	SPATIAL RELATION: 1.0000
	CONDITION DESCRIPTION: 0.3034
	COUNTRY: 1.0000
	TIMERANGE: 0.9609
	CITY: 0.9577
	CONDITION TEMPERATURE: 0.6286
	CURRENT LOCATION: 0.4551
	GEOGRAPHIC POI: 0.8458

	OVERALL: 0.8390



In [108]:
descriptions = {}
with open('/Users/suser/Courses/Semantic Parsing/datasets/SNIPS/slot_descriptions.txt') as inf:
    for line in inf:
        if line.strip() == '':
            continue
        intent = line.strip().split(' ')[0]
        slot = ' '.join(line.strip().split(' ')[1:]).split(':')[0]
        description = ' '.join(line.strip().split(' ')[1:]).split(':')[1].strip()
        
        if intent not in descriptions:
            descriptions[intent] = {}
        
        descriptions[intent][slot] = description

import pprint 
pp = pprint.PrettyPrinter()
pp.pprint(descriptions)

{'AddToPlaylist': {'artist': 'creator of the music item, usually a singer or '
                             'band',
                   'entity name': 'name of the music item or entity',
                   'music item': 'type of music item, usually a song or album',
                   'playlist': 'name of the playlist which typically contains '
                               'a list of songs',
                   'playlist owner': 'person who owns the playlist'},
 'BookRestaurant': {'city': 'location city',
                    'country': 'location country',
                    'cuisine': 'characteristics of food or dishes served at '
                               'the restaurant, usually cultural or regional',
                    'facility': 'facilities or amenities available at the '
                                'restaurant',
                    'party size description': 'description of the people in '
                                              'the dining party',
               

## Final Result Decoding

In [432]:
import pickle
import os
import statistics

domain = 'BookRestaurant'
model = 'context'

data_folder = "/Users/suser/Courses/Semantic Parsing/datasets/SNIPS/processed"
val_data = pickle.load(open(os.path.join(data_folder, 'val_data.p'), 'rb'))

scores = pickle.load(open('beamsearch/ho_{}_ev_{}_{}_bs.p'.format(domain, domain, model), 'rb'))
threshold = 0.1

In [433]:
def check_overlap(spans, span):
    for cspan in spans:
        if span[2][0] >= cspan[2][0] and span[2][0] < cspan[2][1]:
            return True
        if span[2][1] > cspan[2][0] and span[2][1] <= cspan[2][1]:
            return True
        if span[2][0] <= cspan[2][0] and span[2][1] >= cspan[2][1]:
            return True
        if span[0] == cspan[0]:
            return True
    return False

eid = 20
ex = scores[eid]
all_spans = [[en, a[0], a[2], a[1].item()] for en in ex for a in ex[en]]
all_spans.sort(key = lambda x: x[3], reverse=True)
all_spans = [a for a in all_spans if a[3] > threshold]

# print('\n'.join([str(a) for a in all_spans]))

greedy_decode = []
for sp in all_spans:
    if check_overlap(greedy_decode, sp) is False:
        greedy_decode.append(sp)
        
print('\n'.join(['{} ({}) - {:.2f}'.format(a[0].replace(' ','_'), a[1], a[3]) for a in greedy_decode]))

print('\nGold:')
print('\n'.join(['{} ({})'.format(a[1], a[0]) for a in val_data[domain]['entities'][eid]]))

sort (popular) - 1.00
timeRange (5) - 0.87
facility (restaurant) - 0.19

Gold:
sort (popular)
restaurant_type (restaurant)
party_size_number (5)


In [434]:
precision_n = 0
precision_d = 0
recall_n = 0
recall_d = 0

for eid in range(len(scores)):
    ex = scores[eid]
    all_spans = [[en, a[0], a[2], a[1].item()] for en in ex for a in ex[en]]
    all_spans.sort(key = lambda x: x[3], reverse=True)
    all_spans = [a for a in all_spans if a[3] > threshold]

    greedy_decode = []
    for sp in all_spans:
        if check_overlap(greedy_decode, sp) is False:
            greedy_decode.append(sp)
    
    preds = [(a[0].replace(' ','_'), a[1]) for a in greedy_decode]
    gold = [(a[1], a[0]) for a in val_data[domain]['entities'][eid]]
    
    for p in preds:
        if p in gold:
            precision_n += 1
        precision_d += 1
    
    for p in gold:
        if p in preds:
            recall_n += 1
        recall_d += 1

precision = precision_n / precision_d * 100.0
recall = recall_n / recall_d * 100.0
print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1: {:.2f}'.format(statistics.harmonic_mean([precision, recall])))

Precision: 34.82
Recall: 33.96
F1: 34.38


In [508]:
l = [('artist', 'Taylor'), ('playlist', 'rook')]
m = [('playlist', 'rook'), ('artist', 'Taylor')]

[('artist', 'Taylor'), ('playlist', 'rook')]

In [519]:
a = [1,3,4,1,3,4,1]

len([i for i in a if i == 1])

3

# QA Dataset creation

In [146]:
entity_name_dict = {
    "PlayMusic": ["genre", "year", "sort", "service", "music item", "playlist", "album", "artist", "track"],
    "RateBook": ["object name", "rating unit", "best rating", "rating value", "object type", "object select",
                 "object part of series type"],
    "SearchCreativeWork": ["object name", "object type"],
    "GetWeather": ["state", "spatial relation", "condition description", "country", "timeRange", "city",
                   "condition temperature", "current location", "geographic poi"],
    "BookRestaurant": ["state", "spatial relation", "party size number", "sort", "country", "timeRange",
                       "restaurant type", "served dish", "restaurant name", "city", "cuisine", "poi", "facility",
                       "party size description"],
    "SearchScreeningEvent": ["spatial relation", "object type", "timeRange", "movie name", "movie type",
                             "location name", "object location type"],
    "AddToPlaylist": ["music item", "entity name", "playlist", "artist", "playlist owner"]
}

In [149]:
intents = entity_name_dict.keys()

for intent in intents:
    print(intent)
    slots = entity_name_dict[intent]
    print('All slots:', slots)
    loslots = []
    for lointent in intents:
        if lointent != intent:
            loslots += entity_name_dict[lointent]
    prev_seen = []
    for slot in slots:
        if slot in loslots:
            prev_seen.append(slot)
    print('Previously seen:', prev_seen)
    print("{} / {} ({:.2f}%)".format(len(prev_seen), len(slots), len(prev_seen) / len(slots) * 100.0))

PlayMusic
All slots: ['genre', 'year', 'sort', 'service', 'music item', 'playlist', 'album', 'artist', 'track']
Previously seen: ['sort', 'music item', 'playlist', 'artist']
4 / 9 (44.44%)
RateBook
All slots: ['object name', 'rating unit', 'best rating', 'rating value', 'object type', 'object select', 'object part of series type']
Previously seen: ['object name', 'object type']
2 / 7 (28.57%)
SearchCreativeWork
All slots: ['object name', 'object type']
Previously seen: ['object name', 'object type']
2 / 2 (100.00%)
GetWeather
All slots: ['state', 'spatial relation', 'condition description', 'country', 'timeRange', 'city', 'condition temperature', 'current location', 'geographic poi']
Previously seen: ['state', 'spatial relation', 'country', 'timeRange', 'city']
5 / 9 (55.56%)
BookRestaurant
All slots: ['state', 'spatial relation', 'party size number', 'sort', 'country', 'timeRange', 'restaurant type', 'served dish', 'restaurant name', 'city', 'cuisine', 'poi', 'facility', 'party size d

In [5]:
import json
import sys, os
import random

data_folder = "/Users/suser/Courses/Semantic Parsing/datasets/SNIPS"
    
intents = ["PlayMusic", "RateBook", "SearchCreativeWork", "GetWeather", "BookRestaurant", "AddToPlaylist", 
           "SearchScreeningEvent"]
qa_data = {intent: {'train': {'question': [], 'answers': [], 'context': [], 'id': [], 'title': []}, 
                    'validation': {'question': [], 'answers': [], 'context': [], 'id': [], 'title': []}
                   } for intent in intents}

example_id = 0
add_neg = True
neg_percentage = 0.15

for intent in intents:
    train_file = os.path.join(data_folder, intent, 'train_{}_full.json'.format(intent))
    train_json = json.load(open(train_file, 'rb'))
    
    neg_examples = []
    for i in range(len(train_json[intent])):
        text_so_far = ""
        context = ''.join([a['text'] for a in train_json[intent][i]['data']])
        gold_questions = []
        for a in train_json[intent][i]['data']:
            if 'entity' in a:
                start_idx = len(text_so_far)
                question = a['entity'].replace('_', ' ')
                gold_questions.append(question)
                answer = a['text']
                qa_data[intent]['train']['question'].append(question)
                qa_data[intent]['train']['answers'].append({'answer_start': [start_idx], 'text': [answer]})
                qa_data[intent]['train']['context'].append(context)
                qa_data[intent]['train']['id'].append(example_id)
                qa_data[intent]['train']['title'].append(intent)
                example_id += 1
            text_so_far = text_so_far + a['text']
            
        if add_neg:            
            for question in entity_name_dict[intent]:
                if question not in gold_questions:
                    neg_examples.append((question, context, example_id, intent))
                    example_id += 1
    
    random.shuffle(neg_examples)
    for (question, context, example_id, intent) in neg_examples[: int(neg_percentage * len(neg_examples))]:
        qa_data[intent]['train']['question'].append(question)
        qa_data[intent]['train']['answers'].append({'answer_start': [], 'text': []})
        qa_data[intent]['train']['context'].append(context)
        qa_data[intent]['train']['id'].append(example_id)
        qa_data[intent]['train']['title'].append(intent)
                     
    val_file = os.path.join(data_folder, intent, 'validate_{}.json'.format(intent))
    val_json = json.load(open(val_file, 'rb'))
    
    neg_examples = []
    for i in range(len(val_json[intent])):
        text_so_far = ""
        context = ''.join([a['text'] for a in val_json[intent][i]['data']])
        gold_questions = []
        for a in val_json[intent][i]['data']:
            if 'entity' in a:
                start_idx = len(text_so_far)
                question = a['entity'].replace('_', ' ')
                gold_questions.append(question)
                answer = a['text']
                qa_data[intent]['validation']['question'].append(question)
                qa_data[intent]['validation']['answers'].append({'answer_start': [start_idx], 'text': [answer]})
                qa_data[intent]['validation']['context'].append(context)
                qa_data[intent]['validation']['id'].append(example_id)
                qa_data[intent]['validation']['title'].append(intent)
                example_id += 1
                
            text_so_far = text_so_far + a['text']
            
        if add_neg:
            for question in entity_name_dict[intent]:
                if question not in gold_questions:
                    neg_examples.append((question, context, example_id, intent))
                    example_id += 1
                    
    random.shuffle(neg_examples)
    for (question, context, example_id, intent) in neg_examples[: int(neg_percentage * len(neg_examples))]:
        qa_data[intent]['validation']['question'].append(question)
        qa_data[intent]['validation']['answers'].append({'answer_start': [], 'text': []})
        qa_data[intent]['validation']['context'].append(context)
        qa_data[intent]['validation']['id'].append(example_id)
        qa_data[intent]['validation']['title'].append(intent)

In [6]:
import copy

loo_qa_data = {intent: {'train': {'question': [], 'answers': [], 'context': [], 'id': [], 'title': []}, 
                    'validation': {'question': [], 'answers': [], 'context': [], 'id': [], 'title': []}
                   } for intent in intents}

for intent in intents:
    for split in ['train', 'validation']:
        loo_intents = [j for j in intents if j != intent]
        for li in loo_intents:
            loo_qa_data[intent][split]['question'] += copy.deepcopy(qa_data[li][split]['question'])
            loo_qa_data[intent][split]['answers'] += copy.deepcopy(qa_data[li][split]['answers'])
            loo_qa_data[intent][split]['context'] += copy.deepcopy(qa_data[li][split]['context'])
            loo_qa_data[intent][split]['id'] += copy.deepcopy(qa_data[li][split]['id'])
            loo_qa_data[intent][split]['title'] += copy.deepcopy(qa_data[li][split]['title'])

In [7]:
from datasets import Dataset, DatasetDict

qa_dataset = {}
for intent in intents:
    int_dataset = {}
    for split in ['train', 'validation']:
        int_dataset[split] = Dataset.from_dict(qa_data[intent][split])
        
    qa_dataset[intent] = DatasetDict(int_dataset)

In [11]:
qa_dataset

{'PlayMusic': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 6432
     })
     validation: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 310
     })
 }),
 'RateBook': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 8301
     })
     validation: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 416
     })
 }),
 'SearchCreativeWork': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 3492
     })
     validation: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 177
     })
 }),
 'GetWeather': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_row

In [12]:
loo_qa_dataset = {}
for intent in intents:
    int_dataset = {}
    for split in ['train', 'validation']:
        int_dataset[split] = Dataset.from_dict(loo_qa_data[intent][split])
        
    loo_qa_dataset[intent] = DatasetDict(int_dataset)

In [17]:
loo_qa_dataset

{'PlayMusic': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 39657
     })
     validation: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 2007
     })
 }),
 'RateBook': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 37788
     })
     validation: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 1901
     })
 }),
 'SearchCreativeWork': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 42597
     })
     validation: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         num_rows: 2140
     })
 }),
 'GetWeather': DatasetDict({
     train: Dataset({
         features: ['question', 'answers', 'context', 'id', 'title'],
         n

In [14]:
folder_name = 'qa'
if add_neg:
    folder_name = 'qa-neg'

print(folder_name)
for intent in intents:
    loo_qa_dataset[intent].save_to_disk('/Users/suser/Courses/Semantic Parsing/datasets/SNIPS/{}/{}/ood'.format(folder_name, intent))
    qa_dataset[intent].save_to_disk('/Users/suser/Courses/Semantic Parsing/datasets/SNIPS/{}/{}/sd'.format(folder_name, intent))
    


qa-neg


# Baseline model data creation

In [118]:
import pickle

tag_entity_name_dict = {
    "PlayMusic": ["genre", "year", "sort", "service", "music item", "playlist", "album", "artist", "track", "none"],
    "RateBook": ["object name", "rating unit", "best rating", "rating value", "object type", "object select",
                 "object part of series type", "none"],
    "SearchCreativeWork": ["object name", "object type", "none"],
    "GetWeather": ["state", "spatial relation", "condition description", "country", "timeRange", "city",
                   "condition temperature", "current location", "geographic poi", "none"],
    "BookRestaurant": ["state", "spatial relation", "party size number", "sort", "country", "timeRange",
                       "restaurant type", "served dish", "restaurant name", "city", "cuisine", "poi", "facility",
                       "party size description", "none"],
    "SearchScreeningEvent": ["spatial relation", "object type", "timeRange", "movie name", "movie type",
                             "location name", "object location type", "none"],
    "AddToPlaylist": ["music item", "entity name", "playlist", "artist", "playlist owner", "none"]
}

base_train_data = {k : {} for k in intents}
base_val_data = {k: {} for k in intents}

intents = tag_entity_name_dict.keys()

for intent in intents:
    train_file = os.path.join(data_folder, intent, 'train_{}_full.json'.format(intent))
    train_json = json.load(open(train_file, 'rb'))
    train_utterances = []
    train_tag_indices = []
    train_tag_offsets = []
    for i in range(len(train_json[intent])):
        utt = []
        tags = []
        tag_indices = []
        for a in train_json[intent][i]['data']:
            if a['text'].strip() == '':
                continue
            utt += a['text'].strip().split()
            if 'entity' in a:
                tags += [a['entity'].replace('_', ' ')] * len(a['text'].strip().split())
            else:
                tags += ["none"] * len(a['text'].strip().split())
        utt = ' '.join(utt)
        tag_indices = [tag_entity_name_dict[intent].index(t) for t in tags]
        train_tag_indices.append(tag_indices)
        train_utterances.append(utt)
        
        tag_offsets = []
        utt_splits = utt.split()
        for i in range(len(utt_splits)):
            start_idx = len(' '.join(utt_splits[:i])) + 1 if i > 0 else 0
            tag_offsets.append([start_idx, start_idx + len(utt_splits[i])])
        train_tag_offsets.append(tag_offsets)
        
    base_train_data[intent]['utterances'] = train_utterances
    base_train_data[intent]['tag_indices'] = train_tag_indices
    base_train_data[intent]['tag_offsets'] = train_tag_offsets
    
    val_file = os.path.join(data_folder, intent, 'validate_{}.json'.format(intent))
    val_json = json.load(open(val_file, 'rb'))
    val_utterances = []
    val_tag_indices = []
    val_tag_offsets = []
    for i in range(len(val_json[intent])):
        utt = []
        tags = []
        tag_indices = []
        for a in val_json[intent][i]['data']:
            if a['text'].strip() == '':
                continue
            utt += a['text'].strip().split()
            if 'entity' in a:
                tags += [a['entity'].replace('_', ' ')] * len(a['text'].strip().split())
            else:
                tags += ["none"] * len(a['text'].strip().split())
        utt = ' '.join(utt)
        tag_indices = [tag_entity_name_dict[intent].index(t) for t in tags]
        val_tag_indices.append(tag_indices)
        val_utterances.append(utt)
        
        tag_offsets = []
        utt_splits = utt.split()
        for i in range(len(utt_splits)):
            start_idx = len(' '.join(utt_splits[:i])) + 1 if i > 0 else 0
            tag_offsets.append([start_idx, start_idx + len(utt_splits[i])])
        val_tag_offsets.append(tag_offsets)
        
    base_val_data[intent]['utterances'] = val_utterances
    base_val_data[intent]['tag_indices'] = val_tag_indices
    base_val_data[intent]['tag_offsets'] = val_tag_offsets

for intent in intents:
    for i in range(len(base_val_data[intent]['utterances'])):
        utt = base_train_data[intent]['utterances'][i]
        utt_toks = utt.split(' ')
        tags = base_train_data[intent]['tag_indices'][i]
        if len(utt_toks) != len(tags):
            print(utt, utt_toks, tags)

pickle.dump(base_train_data, open(os.path.join(data_folder, 'base_train_data.p'), 'wb'))
pickle.dump(base_val_data, open(os.path.join(data_folder, 'base_val_data.p'), 'wb'))

In [100]:
print(base_train_data['PlayMusic']['tag_indices'][2])
print(base_train_data['PlayMusic']['utterances'][2])

[9, 9, 1, 9, 9, 7, 7, 9, 3, 9]
Play some seventies music by Janne Puurtinen on Youtube .


In [144]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

for intent in intents:
    for i in range(len(base_train_data[intent]['utterances'])):
        utt = base_train_data[intent]['utterances'][i]
        tags = base_train_data[intent]['tag_indices'][i]
        tag_offsets = torch.tensor(base_train_data[intent]['tag_offsets'][i])
        t_utt = tokenizer(utt, return_tensors = "pt", add_special_tokens=False, return_offsets_mapping=True)
        new_offsets = t_utt['offset_mapping'][0]
        td_utt = [tokenizer.decode([a]) for a in t_utt['input_ids'][0]]
        new_tags = []
        for k in range(new_offsets.shape[0]):
            idx = 0
            while new_offsets[k][0] >= tag_offsets[idx][0]:
                idx += 1
                if idx == len(tag_offsets):
                    break
            new_tags.append(tags[idx-1])
        if len(new_tags) != t_utt['input_ids'][0].shape[0]:
            print(' '.join(td_utt))
            print(new_tags)
    

# TOP Dataset Processing

In [1]:
import os

data_prefix = '/Users/suser/Courses/Semantic Parsing/datasets/TOPv2'

domains = ['event', 'messaging', 'music', 'alarm', 'navigation', 'timer', 'reminder', 'weather']
splits = ['train', 'test', 'eval']
full_data = {}
schema = {k: {'intents': [], 'slots': []} for k in domains}
for split in splits:
    full_data[split] = {}
    for domain in domains:
        dom_train_data = []
        with open(os.path.join(data_prefix, '{}_{}.tsv'.format(domain, split))) as inf:
            for line in inf:
                items = line.strip().split('\t')
                dom_train_data.append([items[0], ' '.join(items[1].split()), items[2]])
                for word in items[2].split():
                    if word[:4] == '[IN:':
                        if word[1:] not in schema[domain]['intents']:
                            schema[domain]['intents'].append(word[1:])
                    if word[:4] == '[SL:':
                        if word[1:] not in schema[domain]['slots']:
                            schema[domain]['slots'].append(word[1:])
        full_data[split][domain] = dom_train_data[1:]

In [2]:
full_data['train']['music'][0]

['music',
 'loop that album',
 '[IN:LOOP_MUSIC loop that [SL:MUSIC_TYPE album ] ]']

In [4]:
utt = full_data['train']['music'][60][1]
lf = full_data['train']['music'][60][2]
words = lf.split()
ent_spans = {}
ent_count = 0
ent_list = []
utt_char_count = 0
for word in words:
    if '[' in word:
        ent_id = word[1:] + '_{}'.format(ent_count)
        ent_spans[ent_id] = {'text':'', 'done':False, 'start_char': -1}
        ent_list.append(ent_id)
        ent_count += 1
    elif ']' in word:
        ent_spans[ent_list[-1]]['done'] = True
        ent_spans[ent_list[-1]]['text'] = ent_spans[ent_list[-1]]['text'].strip()
        ent_list = ent_list[:-1]
    else:
        add_space = True
        if utt_char_count < len(utt):
            if utt[utt_char_count] != ' ':
                add_space = False          
        word_to_add = ' {}'.format(word) if add_space else word
        start_char = utt_char_count + 1 if add_space else utt_char_count
        for e in ent_list:
            if not ent_spans[e]['done']:
                ent_spans[e]['text'] += word_to_add
                if ent_spans[e]['start_char'] == -1:
                    ent_spans[e]['start_char'] = start_char
        utt_char_count += len(word_to_add)
print(ent_spans)

{'IN:PLAY_MUSIC_0': {'text': "play Outkast's new song", 'done': True, 'start_char': 0}, 'SL:MUSIC_ARTIST_NAME_1': {'text': 'Outkast', 'done': True, 'start_char': 5}, 'SL:MUSIC_TYPE_2': {'text': 'song', 'done': True, 'start_char': 19}}


In [21]:
import pickle

qa_data = {}

for split in splits:
    qa_data[split] = {}
    for domain in domains:
        qa_examples = []
        for ex in full_data[split][domain]:
            utt = ex[1]
            lf = ex[2]
            words = lf.split()
            ent_spans = {}
            ent_count = {}
            ent_list = []
            utt_char_count = 0
            for word in words:
                if '[' in word:
                    if word[1:] not in ent_count:
                        ent_count[word[1:]] = 0
                    ent_num = ent_count[word[1:]]
                    ent_id = word[1:] + '_{}'.format(ent_num)
                    ent_spans[ent_id] = {'text':'', 'done':False, 'start_char': -1}
                    ent_list.append(ent_id)
                    ent_count[word[1:]] += 1
                elif ']' in word:
                    ent_spans[ent_list[-1]]['done'] = True
                    ent_spans[ent_list[-1]]['text'] = ent_spans[ent_list[-1]]['text'].strip()
                    ent_list = ent_list[:-1]
                else:
                    add_space = True
                    if utt_char_count < len(utt):
                        if utt[utt_char_count] != ' ':
                            add_space = False          
                    word_to_add = ' {}'.format(word) if add_space else word
                    start_char = utt_char_count + 1 if add_space else utt_char_count
                    for e in ent_list:
                        if not ent_spans[e]['done']:
                            ent_spans[e]['text'] += word_to_add
                            if ent_spans[e]['start_char'] == -1:
                                ent_spans[e]['start_char'] = start_char
                    utt_char_count += len(word_to_add)
            qa_examples.append([('_'.join(k.split('_')[:-1]), utt, lf, 
                                 ent_spans[k]['text'], ent_spans[k]['start_char'], int(k.split('_')[-1])) for k in ent_spans])
        qa_data[split][domain] = qa_examples
        
# pickle.dump(qa_data, open(os.path.join(data_prefix, 'qa', 'unprocessed.p'), 'wb'))

In [212]:
from random import shuffle

spis = 50

qa_data = {}
for split in ['train']:
    qa_data[split] = {}
    for domain in domains:
        qa_examples = []
        fsd_examples = full_data[split][domain]
        for tag in schema[domain]['intents'] + schema[domain]['slots']:
            tag_examples = [e for e in fsd_examples if tag in e[2]]
            shuffle(tag_examples)
            for ex in tag_examples[:spis]:
                utt = ex[1]
                lf = ex[2]
                words = lf.split()
                ent_spans = {}
                ent_count = {}
                ent_list = []
                utt_char_count = 0
                for word in words:
                    if '[' in word:
                        if word[1:] not in ent_count:
                            ent_count[word[1:]] = 0
                        ent_num = ent_count[word[1:]]
                        ent_id = word[1:] + '_{}'.format(ent_num)
                        ent_spans[ent_id] = {'text':'', 'done':False, 'start_char': -1}
                        ent_list.append(ent_id)
                        ent_count[word[1:]] += 1
                    elif ']' in word:
                        ent_spans[ent_list[-1]]['done'] = True
                        ent_spans[ent_list[-1]]['text'] = ent_spans[ent_list[-1]]['text'].strip()
                        ent_list = ent_list[:-1]
                    else:
                        add_space = True
                        if utt_char_count < len(utt):
                            if utt[utt_char_count] != ' ':
                                add_space = False          
                        word_to_add = ' {}'.format(word) if add_space else word
                        start_char = utt_char_count + 1 if add_space else utt_char_count
                        for e in ent_list:
                            if not ent_spans[e]['done']:
                                ent_spans[e]['text'] += word_to_add
                                if ent_spans[e]['start_char'] == -1:
                                    ent_spans[e]['start_char'] = start_char
                        utt_char_count += len(word_to_add)
                qa_examples.append([('_'.join(k.split('_')[:-1]), utt, lf, 
                                     ent_spans[k]['text'], ent_spans[k]['start_char'], int(k.split('_')[-1])) for k in ent_spans])
        qa_data[split][domain] = qa_examples

In [24]:
qa_data['train']['alarm'][1000]

[('IN:CREATE_ALARM',
  'Set alarm for 6:00 am',
  '[IN:CREATE_ALARM Set alarm [SL:DATE_TIME for 6 : 00 am ] ]',
  'Set alarm for 6:00 am',
  0,
  0),
 ('SL:DATE_TIME',
  'Set alarm for 6:00 am',
  '[IN:CREATE_ALARM Set alarm [SL:DATE_TIME for 6 : 00 am ] ]',
  'for 6:00 am',
  10,
  0)]

In [214]:
from datasets import Dataset, DatasetDict

def get_question(token):
    if token[:3] == 'IN:':
        return ' '.join(token[3:].lower().split('_') + ['intent'])
    else:
        return ' '.join(token[3:].lower().split('_') + ['slot'])

full_data_dict = {}
example_id = 0
for domain in domains:
    qa_dataset = {'train': {}, 'test': {}, 'eval': {}, 'train_neg': {}, 'eval_neg': {}, 'test_neg': {}}
    for split in splits:
        qa_dataset[split] = {'question': [], 'context': [], 'id': [], 'title': [], 'answers': []}
        qa_dataset[split + '_neg'] = {'question': [], 'context': [], 'id': [], 'title': [], 'answers': []}
        for ex in qa_data[split][domain]:
            if '[IN:UNSUPPORTED_' in ex[0][2]:
                continue
            for sl in ex:
                qa_dataset[split]['question'].append(get_question(sl[0]))
                qa_dataset[split]['context'].append(sl[1])
                qa_dataset[split]['id'].append(example_id)
                qa_dataset[split]['title'].append(domain)
                qa_dataset[split]['answers'].append({'answer_start': [sl[4]], 'text': [sl[3]]})
                example_id += 1
                
            present_tags = [sl[0] for sl in ex]
            for tag in schema[domain]['slots'] + schema[domain]['intents']:
                if tag not in present_tags:
                    qa_dataset[split + '_neg']['question'].append(get_question(tag))
                    qa_dataset[split + '_neg']['context'].append(ex[0][1])
                    qa_dataset[split + '_neg']['id'].append(example_id)
                    qa_dataset[split + '_neg']['title'].append(domain)
                    qa_dataset[split + '_neg']['answers'].append({'answer_start': [], 'text': []})
                    example_id += 1
            
        qa_dataset[split] = Dataset.from_dict(qa_dataset[split])
        qa_dataset[split + '_neg'] = Dataset.from_dict(qa_dataset[split + '_neg'])
        qa_dataset[split + '_neg'] = qa_dataset[split + '_neg'].cast(qa_dataset[split].features)
    full_data_dict[domain] = DatasetDict(qa_dataset)

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/10 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/8 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [219]:
full_data_dict['alarm']

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'id', 'title', 'answers'],
        num_rows: 2068
    })
    train_neg: Dataset({
        features: ['question', 'context', 'id', 'title', 'answers'],
        num_rows: 9894
    })
})

In [216]:
for domain in domains:
    full_data_dict[domain].save_to_disk(os.path.join(data_prefix, 'qa', domain))

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

max_len = 0
for d in domains:
    for split in splits:
        for ex in qa_data[split][domain]:
            for sl in ex:
                if len(tokenizer(sl[1])['input_ids']) > max_len:
                    max_len = len(tokenizer(sl[1])['input_ids'])
                
print(max_len)

26


In [456]:
pickle.dump(schema, open(os.path.join(data_prefix, 'qa', 'schema.p'), 'wb'))

In [11]:
for domain in schema:
    print(domain.upper())
    print('Intents: {}'.format(len(schema[domain]['intents'])))
    print('Slots: {}'.format(len(schema[domain]['slots'])))
    print()

EVENT
Intents: 11
Slots: 17

MESSAGING
Intents: 12
Slots: 27

MUSIC
Intents: 15
Slots: 9

ALARM
Intents: 8
Slots: 9

NAVIGATION
Intents: 17
Slots: 33

TIMER
Intents: 11
Slots: 5

REMINDER
Intents: 19
Slots: 32

WEATHER
Intents: 7
Slots: 11



In [59]:
multi_entity_counts = {d: {} for d in domains}
for d in domains:
    dup_utt = 0
    dup_ent_count = []
    for ex in qa_data['test'][d]:
        if 'UNSUPPORTED' in ex[0][2]:
            continue
        entities = [a[0] for a in ex]
        set_entities = list(set(entities))
        set_counts = {s: 0 for s in set_entities}
        for e in entities:
            set_counts[e] += 1
        
        for e in set_counts:
            if e in multi_entity_counts[d]:
                if set_counts[e] > multi_entity_counts[d][e]:
                    multi_entity_counts[d][e] = set_counts[e]
            else:
                multi_entity_counts[d][e] = set_counts[e]

        max_dup_ent_count = max(list(set_counts.values()))
#         if max_dup_ent_count == 5:
#             print(ex)
#             break
        if max_dup_ent_count > 1:
            dup_utt += 1
            dup_ent_count.append(max_dup_ent_count)
            
    print(d)
    print(dup_utt, len(qa_data['test'][d]))
    print(max(dup_ent_count))

event
235 2654
3
messaging
111 3048
4
music
188 4184
3
alarm
301 7123
5
navigation
368 6075
4
timer
157 4252
4
reminder
695 5767
4
weather
215 5682
2


In [28]:
for d in multi_entity_counts:
    print(d)
    for e in multi_entity_counts[d]:
        print('\t', e, multi_entity_counts[d][e])
    print()
pickle.dump(multi_entity_counts, open(os.path.join(data_prefix, 'qa', 'me_counts.p'), 'wb'))

event
	 SL:DATE_TIME 2
	 IN:GET_EVENT 1
	 SL:CATEGORY_EVENT 2
	 SL:TYPE_RELATION 1
	 IN:GET_CONTACT 1
	 SL:ATTENDEE_EVENT 1
	 SL:CONTACT_RELATED 1
	 SL:NAME_EVENT 1
	 SL:ATTRIBUTE_EVENT 2
	 SL:ORDINAL 1
	 IN:GET_EVENT_ORGANIZER 1
	 SL:ORGANIZER_EVENT 1
	 SL:LOCATION 3
	 SL:LOCATION_USER 1
	 IN:GET_LOCATION 2
	 SL:SEARCH_RADIUS 1
	 SL:POINT_ON_MAP 1
	 SL:LOCATION_MODIFIER 1
	 SL:CATEGORY_LOCATION 1
	 SL:CONTACT 1
	 IN:GET_LOCATION_HOME 1
	 IN:GET_LOCATION_SCHOOL 1
	 IN:NEGATION 1

messaging
	 SL:TAG_MESSAGE 1
	 IN:GET_MESSAGE 1
	 SL:TYPE_REACTION 2
	 SL:RECIPIENT 4
	 IN:REACT_MESSAGE 1
	 IN:SEND_MESSAGE 1
	 SL:CONTENT_EXACT 2
	 SL:TYPE_RELATION 3
	 IN:GET_CONTACT 3
	 SL:CONTACT_RELATED 2
	 SL:ORDINAL 1
	 SL:DATE_TIME 1
	 SL:CONTACT 2
	 SL:AMOUNT 1
	 SL:MUTUAL_SCHOOL 1
	 IN:GET_EVENT_ATTENDEE 1
	 SL:CATEGORY_EVENT 1
	 SL:CONTENT_EMOJI 1
	 IN:GET_EVENT_ORGANIZER 1
	 SL:MUTUAL_EMPLOYER 1
	 SL:LOCATION 1
	 IN:GET_LOCATION 1
	 SL:CATEGORY_LOCATION 1
	 SL:SENDER 1
	 SL:GROUP 1
	 SL:TYPE_CONTE

In [462]:
d_combs = {}
for d in domains:
    combinations = []
    for ex in qa_data['test'][d]:
        spans = {k: [] for k in list(set([a[3] + '##' + str(a[4]) for a in ex]))}

        for s in ex:
            spans[s[3] + '##' + str(s[4])].append(s[0])

        for k in spans:
            spans[k].sort()
            if len(spans[k]) > 1:
                combo = '##'.join(spans[k])
                combinations.append(combo)

    combinations = list(set(combinations))
    print(d)
    print(len(combinations))
    for c in combinations:
        print(c)
    print()
    d_combs[d] = combinations

event
13
IN:GET_LOCATION##SL:LOCATION##SL:LOCATION_USER
IN:GET_LOCATION_HOME##SL:LOCATION
IN:GET_LOCATION##SL:CATEGORY_LOCATION##SL:LOCATION
IN:GET_CONTACT##SL:ATTENDEE_EVENT
IN:GET_LOCATION##SL:LOCATION
IN:NEGATION##SL:ATTRIBUTE_EVENT
IN:GET_LOCATION_SCHOOL##SL:LOCATION
IN:GET_CONTACT##SL:ORGANIZER_EVENT
IN:GET_EVENT##SL:CATEGORY_EVENT
IN:GET_LOCATION##SL:LOCATION##SL:SEARCH_RADIUS
IN:GET_LOCATION##SL:LOCATION##SL:LOCATION_MODIFIER
IN:GET_LOCATION##SL:LOCATION_MODIFIER
IN:GET_LOCATION##SL:LOCATION##SL:POINT_ON_MAP

messaging
10
IN:GET_MESSAGE##SL:RECIPIENT
IN:SELECT_ITEM##SL:RECIPIENT
IN:GET_LOCATION##SL:CATEGORY_LOCATION##SL:LOCATION
IN:GET_EVENT_ORGANIZER##SL:RECIPIENT
IN:GET_CONTACT##SL:RECIPIENT
IN:GET_CONTACT##SL:SENDER
IN:GET_CONTACT##SL:RECIPIENT##SL:TYPE_RELATION
IN:GET_CONTACT##SL:MUTUAL_EMPLOYER##SL:RECIPIENT
IN:GET_EVENT_ATTENDEE##SL:RECIPIENT
IN:GET_CONTACT##SL:SENDER##SL:TYPE_RELATION

music
1
IN:PLAY_MUSIC##SL:MUSIC_ARTIST_NAME

alarm
4
IN:GET_TIME##SL:ALARM_NAME##SL:DAT

In [471]:
allowed_entities = {d: {} for d in domains}

for d in domains:
    for combo in d_combs[d]:
        ents = combo.split('##')
        for e in ents:
            if get_question(e) not in allowed_entities[d]:
                allowed_entities[d][get_question(e)] = []
            allowed_entities[d][get_question(e)] += [get_question(ee) for ee in ents if ee != e]
            allowed_entities[d][get_question(e)] = list(set(allowed_entities[d][get_question(e)]))
            
pickle.dump(allowed_entities, open(os.path.join(data_prefix, 'qa', 'allowed_ents.p'), 'wb'))

In [472]:
allowed_entities

{'event': {'get location intent': ['location user slot',
   'location modifier slot',
   'point on map slot',
   'category location slot',
   'location slot',
   'search radius slot'],
  'location slot': ['location user slot',
   'get location intent',
   'location modifier slot',
   'point on map slot',
   'category location slot',
   'get location home intent',
   'get location school intent',
   'search radius slot'],
  'location user slot': ['location slot', 'get location intent'],
  'get location home intent': ['location slot'],
  'category location slot': ['location slot', 'get location intent'],
  'get contact intent': ['attendee event slot', 'organizer event slot'],
  'attendee event slot': ['get contact intent'],
  'negation intent': ['attribute event slot'],
  'attribute event slot': ['negation intent'],
  'get location school intent': ['location slot'],
  'organizer event slot': ['get contact intent'],
  'get event intent': ['category event slot'],
  'category event slot': [

# TOP Processing for Seq2Seq-PTR model

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
ex = full_data['test']['music'][52]
ex

['music',
 'Never play Nickelback.',
 '[IN:REMOVE_FROM_PLAYLIST_MUSIC Never play [SL:MUSIC_ARTIST_NAME Nickelback ] . ]']

In [4]:
max_ptr = 0
max_tar = 0
for split in ['test', 'train', 'eval']:
    for domain in full_data[split]:
        outf = open(os.path.join(data_prefix, 'seq2seq_un', '{}_{}.tsv'.format(domain, split)), 'w')
        for ex in full_data[split][domain]:
#             if 'IN:UNSUPPORTED_' in ex[2]:
#                 continue
            utt = ""
            utt_toks = []
            tar = []
            words = ex[2].split()
            tags = []
            text_so_far = []
            count = 0

            for w in words:
                if '[IN:' in w or '[SL:' in w:
                    tags.append(w[1:])
                    text_to_tok = ' '.join(text_so_far)
                    if count > 0:
                        text_to_tok = ' ' + text_to_tok
                    if len(text_so_far) > 0:
                        utt += text_to_tok
                        toks = tokenizer.tokenize(text_to_tok)
                        for t in toks:
                            utt_toks.append(t)
                            tar.append('@ptr_' + str(count))
                            count += 1
                        text_so_far = []
                    tar.append(w)
                elif w == ']':
                    end_tag = tags[-1] + ']'
                    tags = tags[:-1]
                    text_to_tok = ' '.join(text_so_far)
                    if count > 0:
                        text_to_tok = ' ' + text_to_tok
                    if len(text_so_far) > 0:
                        utt += text_to_tok
                        toks = tokenizer.tokenize(text_to_tok)
                        for t in toks:
                            utt_toks.append(t)
                            tar.append('@ptr_' + str(count))
                            count += 1
                        text_so_far = []
                    tar.append(end_tag)
                else:
                    text_so_far.append(w)

            tar_length = len(tar)
            if tar_length > max_tar:
                max_tar = tar_length
            tar = ' '.join(tar)
            outf.write(utt + '\t' + ' '.join(utt_toks) + '\t' + tar + '\n')
            if count > max_ptr:
                max_ptr = count
        outf.close()

In [5]:
max_tar

69

In [8]:
utt

'on Monday what are you forecasting'

In [9]:
tar

'[IN:GET_WEATHER [SL:DATE_TIME @ptr_0 @ptr_1 SL:DATE_TIME] @ptr_2 @ptr_3 @ptr_4 @ptr_5 IN:GET_WEATHER]'

In [10]:
utt_toks

['on', 'ĠMonday', 'Ġwhat', 'Ġare', 'Ġyou', 'Ġforecasting']

In [11]:
tokenizer([utt, 'I play'], return_tensors="pt", padding=True)

{'input_ids': tensor([[    0,   261,   302,    99,    32,    47, 17746,     2],
        [    0,   100,   310,     2,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0]])}

In [2]:
s2s_schema = {}
for d in schema:
    s2s_schema[d] = {'intents': [], 'slots': []}
    for i in schema[d]['intents']:
#         if 'IN:UNSUPPORTED' in i:
#             continue
        s2s_schema[d]['intents'].append('[' + i)
        s2s_schema[d]['intents'].append(i + ']')
    for i in schema[d]['slots']:
        s2s_schema[d]['slots'].append('[' + i)
        s2s_schema[d]['slots'].append(i + ']')

In [4]:
import pickle

pickle.dump(s2s_schema, open(os.path.join(data_prefix, 'seq2seq_un', 'schema.p'), 'wb'))

In [20]:
for d in schema:
    for i in schema[d]['intents']:
        print(d, i)
    for i in schema[d]['slots']:
        print(d, i)

event IN:GET_EVENT
event IN:GET_LOCATION
event IN:GET_EVENT_ATTENDEE
event IN:UNSUPPORTED_EVENT
event IN:GET_LOCATION_HOME
event IN:GET_LOCATION_SCHOOL
event IN:GET_CONTACT
event IN:GET_LOCATION_WORK
event IN:NEGATION
event IN:GET_EVENT_ORGANIZER
event IN:GET_EVENT_ATTENDEE_AMOUNT
event SL:LOCATION
event SL:POINT_ON_MAP
event SL:CATEGORY_EVENT
event SL:DATE_TIME
event SL:SEARCH_RADIUS
event SL:LOCATION_USER
event SL:ATTRIBUTE_EVENT
event SL:ATTENDEE_EVENT
event SL:LOCATION_MODIFIER
event SL:CATEGORY_LOCATION
event SL:NAME_EVENT
event SL:ORDINAL
event SL:CONTACT
event SL:CONTACT_RELATED
event SL:TYPE_RELATION
event SL:ORGANIZER_EVENT
event SL:AMOUNT
messaging IN:GET_MESSAGE
messaging IN:SEND_MESSAGE
messaging IN:GET_CONTACT
messaging IN:REACT_MESSAGE
messaging IN:CANCEL_MESSAGE
messaging IN:GET_EVENT_ATTENDEE
messaging IN:UNSUPPORTED_MESSAGING
messaging IN:IGNORE_MESSAGE
messaging IN:SEND_TEXT_MESSAGE
messaging IN:GET_EVENT_ORGANIZER
messaging IN:GET_LOCATION
messaging IN:SELECT_ITEM
me

# TOP Seq2Seq Span Pointer Processing

In [152]:
max_tar = 0

for split in ['test', 'train', 'eval']:
    for domain in full_data[split]:
        outf = open(os.path.join(data_prefix, 'seq2seq_sp', '{}_{}.tsv'.format(domain, split)), 'w')
        for ex in full_data[split][domain]:
            if 'IN:UNSUPPORTED_' in ex[2]:
                continue
            utt = ""
            utt_toks = []
            tar = []
            words = ex[2].split()
            tags = []
            text_so_far = []
            count = 0

            for w in words:
                if '[IN:' in w or '[SL:' in w:
                    tags.append(w[1:])
                    text_to_tok = ' '.join(text_so_far)
                    if count > 0:
                        text_to_tok = ' ' + text_to_tok
                    if len(text_so_far) > 0:
                        utt += text_to_tok
                        toks = tokenizer.tokenize(text_to_tok)
                        for t in toks:
                            utt_toks.append(t)
                            tar.append('@ptr_' + str(count))
                            count += 1
                        text_so_far = []
                    tar.append(w)
                elif w == ']':
                    end_tag = tags[-1] + ']'
                    tags = tags[:-1]
                    text_to_tok = ' '.join(text_so_far)
                    if count > 0:
                        text_to_tok = ' ' + text_to_tok
                    if len(text_so_far) > 0:
                        utt += text_to_tok
                        toks = tokenizer.tokenize(text_to_tok)
                        for t in toks:
                            utt_toks.append(t)
                            tar.append('@ptr_' + str(count))
                            count += 1
                        text_so_far = []
                    tar.append(end_tag)
                else:
                    text_so_far.append(w)

            ent_list = []
            new_tar = []
            for token in tar:
                if '[' in token:
                    ent_list.append(token[1:])
                    new_tar.append(token)
                elif ']' in token:
                    ent_list = ent_list[:-1]
                    new_tar.append(token)
                else:
                    if 'SL:' in ent_list[-1]:
                        new_tar.append(token)
                        
            redundant_ptr = [False]
            for i in range(1, len(new_tar) - 1):
                if '@ptr' in new_tar[i]:
                    if ('@ptr' in new_tar[i-1]) and ('@ptr' in new_tar[i+1]):
                        redundant_ptr.append(True)
                    else:
                        redundant_ptr.append(False)
                else:
                    redundant_ptr.append(False)
            redundant_ptr.append(False)
            newr_tar = []
            for i in range(len(new_tar)):
                if not redundant_ptr[i]:
                    newr_tar.append(new_tar[i])
            
            repeat_ptr = [False]
            for i in range(1, len(newr_tar) - 1):
                if '@ptr' in newr_tar[i]:
                    if ('@ptr' not in newr_tar[i-1]) and ('@ptr' not in newr_tar[i+1]):
                        repeat_ptr.append(True)
                    else:
                        repeat_ptr.append(False)
                else:
                    repeat_ptr.append(False)
            repeat_ptr.append(False)
            newrr_tar = []
            for i in range(len(newr_tar)):
                newrr_tar.append(newr_tar[i])
                if repeat_ptr[i]:
                    newrr_tar.append(newr_tar[i])
                    
            tar_length = len(newrr_tar)
            if tar_length > max_tar:
                max_tar = tar_length
            tar = ' '.join(newrr_tar)
            outf.write(utt + '\t' + ' '.join(utt_toks) + '\t' + tar + '\n')
        outf.close()

['Rem', 'ind', 'ĠDad', 'Ġ,', 'ĠUncle', 'ĠCharles', 'Ġ,', 'ĠUncle', 'ĠJimmy', 'Ġ,', 'ĠJason', 'Ġand', 'Ġmyself', 'Ġon', 'ĠFriday', 'Ġnight', 'Ġto', 'Ġpack', 'Ġfor', 'Ġour', 'Ġfishing', 'Ġtrip', 'Ġon', 'ĠSaturday', 'Ġmorning', 'Ġ.']
[IN:CREATE_REMINDER [SL:PERSON_REMINDED [IN:GET_CONTACT [SL:TYPE_RELATION @ptr_2 @ptr_2 SL:TYPE_RELATION] IN:GET_CONTACT] SL:PERSON_REMINDED] [SL:PERSON_REMINDED [IN:GET_CONTACT [SL:TYPE_RELATION @ptr_4 @ptr_4 SL:TYPE_RELATION] [SL:CONTACT @ptr_5 @ptr_5 SL:CONTACT] IN:GET_CONTACT] SL:PERSON_REMINDED] [SL:PERSON_REMINDED [IN:GET_CONTACT [SL:TYPE_RELATION @ptr_7 @ptr_7 SL:TYPE_RELATION] [SL:CONTACT @ptr_8 @ptr_8 SL:CONTACT] IN:GET_CONTACT] SL:PERSON_REMINDED] [SL:PERSON_REMINDED @ptr_10 @ptr_10 SL:PERSON_REMINDED] [SL:PERSON_REMINDED @ptr_12 @ptr_12 SL:PERSON_REMINDED] [SL:DATE_TIME @ptr_13 @ptr_15 SL:DATE_TIME] [SL:TODO [IN:GET_TODO [SL:TODO @ptr_17 @ptr_21 SL:TODO] [SL:DATE_TIME @ptr_22 @ptr_24 SL:DATE_TIME] IN:GET_TODO] SL:TODO] IN:CREATE_REMINDER]


In [151]:
max_tar

58

# SPIS Processing

In [34]:
from random import shuffle
import os

spis = 25

data_prefix = '/Users/suser/Courses/Semantic Parsing/datasets/TOPv2'
domains = ['event', 'messaging', 'music', 'alarm', 'navigation', 'timer', 'reminder', 'weather']

for split in ['train']:
    for domain in domains:
        inf = open(os.path.join(data_prefix, 'seq2seq_un', '{}_{}.tsv'.format(domain, split)), 'r')
        examples = []
        schema_tags = {}
        for line in inf:
            lex = line.strip().split('\t')
            for word in lex[2].split():
                if word[:4] == '[IN:' or word[:4] == '[SL:':
                    schema_tags[word] = 1
            examples.append(lex)
        inf.close()
        outf = open(os.path.join(data_prefix, 'lr/s3', 'spi_{}'.format(spis), '{}_{}.tsv'.format(domain, split)), 'w')
        sel_examples = []
        for tag in schema_tags:
            tag_examples = [e for e in examples if tag in e[2]]
            shuffle(tag_examples)
            if len(tag_examples) < spis:
                print('Not enough examples in {}/{}/{} - {}'.format(domain, split, tag, len(tag_examples)))
            sel_count = 0
            for ex in tag_examples:
                if ex not in sel_examples:
                    sel_examples.append('\t'.join(ex))
                    sel_count += 1
                if sel_count == spis:
                    break
        for sel_ex in sel_examples:
            outf.write(sel_ex + '\n')
        print('Done with {}/{}'.format(split, domain))
        outf.close()

Not enough examples in event/train/[IN:GET_EVENT_ATTENDEE - 8
Not enough examples in event/train/[IN:GET_LOCATION_SCHOOL - 2
Not enough examples in event/train/[IN:GET_CONTACT - 19
Not enough examples in event/train/[SL:CONTACT_RELATED - 18
Not enough examples in event/train/[SL:TYPE_RELATION - 20
Not enough examples in event/train/[SL:ORGANIZER_EVENT - 21
Not enough examples in event/train/[IN:GET_LOCATION_WORK - 5
Not enough examples in event/train/[IN:NEGATION - 1
Not enough examples in event/train/[IN:GET_EVENT_ORGANIZER - 4
Not enough examples in event/train/[IN:GET_EVENT_ATTENDEE_AMOUNT - 4
Not enough examples in event/train/[SL:AMOUNT - 1
Done with train/event
Not enough examples in messaging/train/[SL:LOCATION_HOME - 6
Not enough examples in messaging/train/[IN:GET_EVENT_ATTENDEE - 1
Not enough examples in messaging/train/[SL:TYPE_CONTACT - 3
Not enough examples in messaging/train/[SL:MUTUAL_EMPLOYER - 12
Not enough examples in messaging/train/[SL:MUTUAL_SCHOOL - 4
Not enough e

In [50]:
schema_spans = {}
for d in schema:
    schema_spans[d] = {}
    for t in schema[d]['intents']:
        schema_spans[d][t] = []
    for t in schema[d]['slots']:
        schema_spans[d][t] = []

In [51]:
for d in domains:
    for example in qa_data['train'][d]:
        for tag in example:
            schema_spans[d][tag[0]].append(tag[3])
    for example in qa_data['eval'][d]:
        for tag in example:
            schema_spans[d][tag[0]].append(tag[3])
    for example in qa_data['test'][d]:
        for tag in example:
            schema_spans[d][tag[0]].append(tag[3])

In [52]:
for d in schema_spans:
    for t in schema_spans[d]:
        ol = len(schema_spans[d][t])
        schema_spans[d][t] = list(set([i.lower() for i in schema_spans[d][t]]))
        print(d, t, ol, len(schema_spans[d][t]))
        if len(schema_spans[d][t]) < 5:
            print(schema_spans[d][t])

event IN:GET_EVENT 12726 12171
event IN:GET_LOCATION 2564 928
event IN:GET_EVENT_ATTENDEE 4 4
['did sarah go to the 20 year reunion?', 'is jack going to the concert', 'friends who have responded to my event on sunday', 'are any of my friends going to the concert']
event IN:UNSUPPORTED_EVENT 424 423
event IN:GET_LOCATION_HOME 116 19
event IN:GET_LOCATION_SCHOOL 3 1
['school']
event IN:GET_CONTACT 23 11
event IN:GET_LOCATION_WORK 7 2
['my office', 'work']
event IN:NEGATION 2 2
['gambling', 'alcohol']
event IN:GET_EVENT_ORGANIZER 5 5
event IN:GET_EVENT_ATTENDEE_AMOUNT 4 4
['how many people have purchased tickets for the rhiannon concert in atlanta this saturday night', 'how many people expected at the amphitheatre', 'how many people will be attending the event', 'how many people expected to show up on first star wars showing']
event SL:LOCATION 9030 2185
event SL:POINT_ON_MAP 688 517
event SL:CATEGORY_EVENT 7609 2456
event SL:DATE_TIME 9980 888
event SL:SEARCH_RADIUS 1183 38
event SL:LOCA

In [53]:
schema_spans['timer']['SL:METHOD_TIMER']

['timeout timer',
 'timeout',
 'counter',
 'buzzer',
 'my timer',
 'times',
 'timer countdown',
 'clock',
 'count',
 'timer',
 'countdown',
 'count down',
 'the countdown',
 'timing',
 'stopwatch',
 'time out timer',
 'stopwatch timer',
 'timer suspend',
 'coutndown',
 'countdown timer',
 'time out',
 'timer.',
 'snooze',
 'time',
 'thirty minute',
 'my',
 '2 hours',
 'timers',
 'the timer']

# SNIPS Seq2seq model

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [19]:
import sys, os
import json

data_folder = "/Users/suser/Courses/Semantic Parsing/datasets/SNIPS/seq2seq"

intents = ["PlayMusic", "RateBook", "SearchCreativeWork", "GetWeather", "BookRestaurant", "AddToPlaylist", "SearchScreeningEvent"]
train_data = {k : {} for k in intents}
val_data = {k: {} for k in intents}

for intent in intents:
    train_file = os.path.join(data_folder, 'train_{}_full.json'.format(intent))
    train_json = json.load(open(train_file, 'rb'))
    train_utterances = [''.join([a['text'] for a in train_json[intent][i]['data']]) for i in range(len(train_json[intent]))]
    
    train_lfs = []
    for i in range(len(train_json[intent])):
        lf = ""
        for a in train_json[intent][i]['data']:
            if 'entity' in a:
                lf +=  ' [SL:' + a['entity'] + ' ' + a['text'] + ' ' + 'SL:' + a['entity'] + ']'
            else:
                lf += ' ' + a['text']
        train_lfs.append(' '.join([a for a in lf.split() if a != '']))
    
    train_data[intent]['utterances'] = train_utterances
    train_data[intent]['lfs'] = train_lfs
    
    val_file = os.path.join(data_folder, 'validate_{}.json'.format(intent))
    val_json = json.load(open(val_file, 'rb'))
    val_utterances = [''.join([a['text'] for a in val_json[intent][i]['data']]) for i in range(len(val_json[intent]))]
    
    val_lfs = []
    for i in range(len(val_json[intent])):
        lf = ""
        for a in val_json[intent][i]['data']:
            if 'entity' in a:
                lf +=  ' [SL:' + a['entity'] + ' ' + a['text'] + ' ' + 'SL:' + a['entity'] + ']'
            else:
                lf += ' ' + a['text']
        val_lfs.append(' '.join([a for a in lf.split() if a != '']))
        
    val_data[intent]['utterances'] = val_utterances
    val_data[intent]['lfs'] = val_lfs
    
data = {'train': train_data, 'eval': val_data}
    

In [20]:
data['train']['PlayMusic']['lfs'][:10]

['I need to hear the [SL:music_item song SL:music_item] [SL:track Aspro Mavro SL:track] from [SL:artist Bill Szymczyk SL:artist] on [SL:service Youtube SL:service]',
 'play [SL:track Yo Ho SL:track] from [SL:artist the new york pops SL:artist] on [SL:service Youtube SL:service]',
 'Play some [SL:year seventies SL:year] music by [SL:artist Janne Puurtinen SL:artist] on [SL:service Youtube SL:service] .',
 'play the [SL:album Música Da Série De Filmes O Hobbit SL:album] [SL:music_item album SL:music_item] by [SL:artist Alex Otaola SL:artist]',
 'Play [SL:artist Magic Sam SL:artist] from the [SL:year thirties SL:year]',
 'Play [SL:track The Soft Parade SL:track] .',
 'Play music by [SL:artist blowfly SL:artist] from the [SL:year seventies SL:year]',
 'play [SL:artist Jeff Pilson SL:artist] on [SL:service Youtube SL:service]',
 'play a [SL:music_item tune SL:music_item] by [SL:artist Syreeta Wright SL:artist] from [SL:year twenties SL:year] from the [SL:sort top SL:sort]',
 "Please play me

In [26]:
max_ptr = 0
max_tar = 0

schema = {d: {'intents':[], 'slots':[]} for d in data['train']}

for split in ['train', 'eval']:
    for domain in data[split]:
        outf = open(os.path.join(data_folder, '{}_{}.tsv'.format(domain, split)), 'w')
        for ex in data[split][domain]['lfs']:
            utt = ""
            utt_toks = []
            tar = []
            words = ex.split()
            text_so_far = []
            count = 0

            for w in words:
                if '[SL:' in w:
                    if w not in schema[domain]['slots']:
                        schema[domain]['slots'].append(w)
                    text_to_tok = ' '.join(text_so_far)
                    if count > 0:
                        text_to_tok = ' ' + text_to_tok
                    if len(text_so_far) > 0:
                        utt += text_to_tok
                        toks = tokenizer.tokenize(text_to_tok)
                        for t in toks:
                            utt_toks.append(t)
                            tar.append('@ptr_' + str(count))
                            count += 1
                        text_so_far = []
                    tar.append(w)
                elif 'SL:' in w and ']' in w:
                    if w not in schema[domain]['slots']:
                        schema[domain]['slots'].append(w)
                    text_to_tok = ' '.join(text_so_far)
                    if count > 0:
                        text_to_tok = ' ' + text_to_tok
                    if len(text_so_far) > 0:
                        utt += text_to_tok
                        toks = tokenizer.tokenize(text_to_tok)
                        for t in toks:
                            utt_toks.append(t)
                            tar.append('@ptr_' + str(count))
                            count += 1
                        text_so_far = []
                    tar.append(w)
                else:
                    text_so_far.append(w)

            tar_length = len(tar)
            if tar_length > max_tar:
                max_tar = tar_length
            tar = ' '.join(tar)
            outf.write(utt + '\t' + ' '.join(utt_toks) + '\t' + tar + '\n')
            if count > max_ptr:
                max_ptr = count
        outf.close()

In [29]:
import pickle

pickle.dump(schema, open(os.path.join(data_folder, 'schema.p'), 'wb'))

In [27]:
from random import shuffle
import os

spis = 1

data_prefix = '/Users/suser/Courses/Semantic Parsing/datasets/SNIPS'
domains = ["PlayMusic", "RateBook", "SearchCreativeWork", "GetWeather", "BookRestaurant", "AddToPlaylist", "SearchScreeningEvent"]

for split in ['train']:
    for domain in domains:
        inf = open(os.path.join(data_prefix, 'seq2seq', '{}_{}.tsv'.format(domain, split)), 'r')
        examples = []
        schema_tags = {}
        for line in inf:
            lex = line.strip().split('\t')
            for word in lex[2].split():
                if word[:4] == '[IN:' or word[:4] == '[SL:':
                    schema_tags[word] = 1
            examples.append(lex)
        inf.close()
        outf = open(os.path.join(data_prefix, 'lr/s3', 'spi_{}'.format(spis), '{}_{}.tsv'.format(domain, split)), 'w')
        sel_examples = []
        for tag in schema_tags:
            tag_examples = [e for e in examples if tag in e[2]]
            shuffle(tag_examples)
            if len(tag_examples) < spis:
                print('Not enough examples in {}/{}/{} - {}'.format(domain, split, tag, len(tag_examples)))
            sel_count = 0
            for ex in tag_examples:
                if ex not in sel_examples:
                    sel_examples.append('\t'.join(ex))
                    sel_count += 1
                if sel_count == spis:
                    break
        for sel_ex in sel_examples:
            outf.write(sel_ex + '\n')
        print('Done with {}/{}'.format(split, domain))
        outf.close()

Done with train/PlayMusic
Done with train/RateBook
Done with train/SearchCreativeWork
Done with train/GetWeather
Done with train/BookRestaurant
Done with train/AddToPlaylist
Done with train/SearchScreeningEvent


In [36]:
for d in schema:
    print(d, len(schema[d]['slots']) / 2)

PlayMusic 9.0
RateBook 7.0
SearchCreativeWork 2.0
GetWeather 9.0
BookRestaurant 14.0
AddToPlaylist 5.0
SearchScreeningEvent 7.0


In [49]:
pred = "Turn off [SL:AMOUNT all SL:AMOUNT] alarms [SL:ALARM_NAME today SL:ALARM_NAME]"

def acc_process(parse):
    words = parse.split(' ')
    tags = []
    processed = []
    for word in words:
        if '[' in word:
            tags.append(word)
            processed.append(word)
        elif ']' in word:
            tags = tags[:-1]
            processed.append(']')
        else:
            if len(tags) > 0:
                if 'SL:' in tags[-1]:
                    processed.append(word)
    return ' '.join(processed)

print(pred)
print(acc_process(pred))


Turn off [SL:AMOUNT all SL:AMOUNT] alarms [SL:ALARM_NAME today SL:ALARM_NAME]
[SL:AMOUNT all ] [SL:ALARM_NAME today ]


# Partial Loading Testing

In [7]:
import torch
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer

class S2S(torch.nn.Module):
    def __init__(self, enc, dec):
        super(S2S, self).__init__()
        self.enc = enc
        self.dec = dec
        
    def forward(self, x):
        return self.dec(self.enc(x))

enc = TransformerEncoder(TransformerEncoderLayer(d_model=10, nhead=1, batch_first=True), num_layers=1)
dec = TransformerDecoder(TransformerDecoderLayer(d_model=10, nhead=1, batch_first=True), num_layers=1)

model = S2S(enc, dec)
state_dict = {'model_state_dict': model.state_dict()}
torch.save(state_dict, 'part_model.pt')

In [11]:
all_keys = [k for k in state_dict['model_state_dict']]
for k in all_keys:
    if k[:3] == 'dec':
        state_dict['model_state_dict'].pop(k)

In [15]:
n_model = S2S(enc, dec)
n_state_dict = {'model_state_dict': n_model.state_dict()}

{'model_state_dict': OrderedDict([('enc.layers.0.self_attn.in_proj_weight',
               tensor([[-9.0695e-02, -1.5757e-02, -3.6333e-01,  2.8670e-01, -3.5121e-01,
                        -3.4356e-01,  1.7307e-01,  3.5649e-01, -1.9847e-01,  2.3455e-01],
                       [ 2.5648e-01, -2.0569e-01,  2.0926e-01,  1.4359e-01, -1.9568e-01,
                        -2.2022e-01, -2.3071e-03,  4.7106e-02,  2.9593e-01, -1.7319e-01],
                       [-1.2736e-01, -1.5019e-01, -1.8719e-01, -2.1386e-01,  1.3363e-02,
                         5.0641e-02,  3.9216e-02, -1.9087e-01,  3.6287e-01,  8.3013e-02],
                       [ 2.0972e-01,  2.2930e-02, -2.3920e-01,  2.5184e-01, -2.3365e-02,
                        -4.1940e-02,  2.0992e-01, -2.1119e-01, -1.0669e-01,  2.0568e-01],
                       [-1.4517e-01, -2.4690e-02, -3.4193e-01, -3.2237e-01,  1.8344e-01,
                         1.9267e-01,  8.2283e-02,  1.5992e-01, -1.5691e-01,  4.5991e-02],
                       [ 1.29

In [14]:
state_dict

{'model_state_dict': OrderedDict([('enc.layers.0.self_attn.in_proj_weight',
               tensor([[-9.0695e-02, -1.5757e-02, -3.6333e-01,  2.8670e-01, -3.5121e-01,
                        -3.4356e-01,  1.7307e-01,  3.5649e-01, -1.9847e-01,  2.3455e-01],
                       [ 2.5648e-01, -2.0569e-01,  2.0926e-01,  1.4359e-01, -1.9568e-01,
                        -2.2022e-01, -2.3071e-03,  4.7106e-02,  2.9593e-01, -1.7319e-01],
                       [-1.2736e-01, -1.5019e-01, -1.8719e-01, -2.1386e-01,  1.3363e-02,
                         5.0641e-02,  3.9216e-02, -1.9087e-01,  3.6287e-01,  8.3013e-02],
                       [ 2.0972e-01,  2.2930e-02, -2.3920e-01,  2.5184e-01, -2.3365e-02,
                        -4.1940e-02,  2.0992e-01, -2.1119e-01, -1.0669e-01,  2.0568e-01],
                       [-1.4517e-01, -2.4690e-02, -3.4193e-01, -3.2237e-01,  1.8344e-01,
                         1.9267e-01,  8.2283e-02,  1.5992e-01, -1.5691e-01,  4.5991e-02],
                       [ 1.29

In [17]:
n_model.load_state_dict(state_dict['model_state_dict'], strict=False)

_IncompatibleKeys(missing_keys=['dec.layers.0.self_attn.in_proj_weight', 'dec.layers.0.self_attn.in_proj_bias', 'dec.layers.0.self_attn.out_proj.weight', 'dec.layers.0.self_attn.out_proj.bias', 'dec.layers.0.multihead_attn.in_proj_weight', 'dec.layers.0.multihead_attn.in_proj_bias', 'dec.layers.0.multihead_attn.out_proj.weight', 'dec.layers.0.multihead_attn.out_proj.bias', 'dec.layers.0.linear1.weight', 'dec.layers.0.linear1.bias', 'dec.layers.0.linear2.weight', 'dec.layers.0.linear2.bias', 'dec.layers.0.norm1.weight', 'dec.layers.0.norm1.bias', 'dec.layers.0.norm2.weight', 'dec.layers.0.norm2.bias', 'dec.layers.0.norm3.weight', 'dec.layers.0.norm3.bias'], unexpected_keys=[])

In [19]:
n_model.state_dict()

OrderedDict([('enc.layers.0.self_attn.in_proj_weight',
              tensor([[-9.0695e-02, -1.5757e-02, -3.6333e-01,  2.8670e-01, -3.5121e-01,
                       -3.4356e-01,  1.7307e-01,  3.5649e-01, -1.9847e-01,  2.3455e-01],
                      [ 2.5648e-01, -2.0569e-01,  2.0926e-01,  1.4359e-01, -1.9568e-01,
                       -2.2022e-01, -2.3071e-03,  4.7106e-02,  2.9593e-01, -1.7319e-01],
                      [-1.2736e-01, -1.5019e-01, -1.8719e-01, -2.1386e-01,  1.3363e-02,
                        5.0641e-02,  3.9216e-02, -1.9087e-01,  3.6287e-01,  8.3013e-02],
                      [ 2.0972e-01,  2.2930e-02, -2.3920e-01,  2.5184e-01, -2.3365e-02,
                       -4.1940e-02,  2.0992e-01, -2.1119e-01, -1.0669e-01,  2.0568e-01],
                      [-1.4517e-01, -2.4690e-02, -3.4193e-01, -3.2237e-01,  1.8344e-01,
                        1.9267e-01,  8.2283e-02,  1.5992e-01, -1.5691e-01,  4.5991e-02],
                      [ 1.2923e-01,  4.4642e-02, -7.2384e-02